## GRACE/GRACE-FO Harmonic Processing Program

This notebook uses standard Python tools to demonstrate some basic visualization of the Gravity Recovery and Climate Experiment (GRACE) and the GRACE Follow-On (GRACE-FO) Level-2 spherical harmonic products.

The primary and secondary instrumentation onboard the GRACE/GRACE-FO satellites are the ranging instrument (GRACE has a microwave ranging instrument, GRACE-FO has both a microwave ranging instrument and a laser interferometer), the global positioning system (GPS), the accelerometers and the star cameras.
Data from these instruments are combined to estimate the distance between the two satellites, the positions of the satellites in space, the pointing vector of the satellites and any non-gravitational accelerations the satellites experience.
These measurements combined with background gravity field estimates, atmospheric and oceanic variability estimates and tidal estimates are used to create the [Level-2 spherical harmonic product of GRACE and GRACE-FO](https://podaac-tools.jpl.nasa.gov/drive/files/GeodeticsGravity/gracefo/docs/GRACE-FO_L2-UserHandbook_v1.1.pdf).  

There are three main processing centers that create the Level-2 spherical harmonic data as part of the GRACE/GRACE-FO Science Data System (SDS): the [University of Texas Center for Space Research (CSR)](http://www2.csr.utexas.edu/grace/), the [German Research Centre for Geosciences (GeoForschungsZentrum, GFZ)](https://www.gfz-potsdam.de/en/grace/) and the [Jet Propulsion Laboratory (JPL)](https://grace.jpl.nasa.gov/).  

The data is freely available in the US from the [NASA Physical Oceanography Distributed Active Archive Center (PO.DAAC)](https://podaac.jpl.nasa.gov/grace) and internationally from the [GFZ Information System and Data Center](http://isdc.gfz-potsdam.de/grace-isdc/).
There are programs within this repository that can sync with both of these data archives `podaac_grace_sync.py` and `gfz_isdc_grace_ftp.py`.  

This notebook uses Jupyter widgets to set parameters for converting the GRACE/GRACE-FO Level-2 product into monthly spatial maps.
The widgets can be installed as described below.  
```bash
pip3 install --user ipywidgets
jupyter nbextension enable --py --user widgetsnbextension
jupyter-notebook
```

Some of the text included within this notebook about the data processing details comes from [Sutterley and Velicogna (2019)](https://doi.org/10.3390/rs11182108) and [Sutterley et al. (2020)](https://doi.org/10.1029/2019EA000860).

### Load necessary modules for running the notebook

In [ ]:
import os
import numpy as np
import matplotlib
matplotlib.rcParams['mathtext.default'] = 'regular'
matplotlib.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams["animation.embed_limit"] = 40
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.animation as animation
import cartopy.crs as ccrs
import ipywidgets as widgets
from IPython.display import HTML, Latex

from gravity_toolkit.grace_find_months import grace_find_months
from gravity_toolkit.grace_input_months import grace_input_months
from gravity_toolkit.read_GIA_model import read_GIA_model
from gravity_toolkit.harmonics import harmonics
from gravity_toolkit.spatial import spatial
from gravity_toolkit.read_love_numbers import read_love_numbers
from gravity_toolkit.units import units
from gravity_toolkit.plm_holmes import plm_holmes
from gravity_toolkit.gauss_weights import gauss_weights
from gravity_toolkit.ocean_stokes import ocean_stokes
from gravity_toolkit.harmonic_summation import harmonic_summation
from gravity_toolkit.ncdf_write import ncdf_write
from gravity_toolkit.hdf5_write import hdf5_write
from gravity_toolkit.utilities import get_data_path

### Set the GRACE/GRACE-FO Data Directory
Directory should contain:  
- Technical notes with SLR coefficients  
- Subdirectory with geocenter coefficients  
- Subdirectories for each processing center  

In [ ]:
# set the directory with GRACE/GRACE-FO data
dirText = widgets.Text(
    value=os.getcwd(),
    description='Directory:',
    disabled=False
)
display(dirText)

### Update Data in Directory

In [ ]:
# extract directory value from widget
base_dir = os.path.expanduser(dirText.value)
# run podaac sync program to get latest data
!podaac_grace_sync.py --directory=$base_dir --netrc=~/.netrc
# run GRACE date program to verify months
!run_grace_date.py --directory=$base_dir --verbose

### Set GRACE/GRACE-FO Parameters
These parameters describe the specific GRACE/GRACE-FO product and the months of data to read  

- GRACE/GRACE-FO Processing Center
    * CSR: University of Texas Center for Space Research  
    * GFZ: German Research Centre for Geosciences (GeoForschungsZentrum)
    * JPL: Jet Propulsion Laboratory    
    * CNES: French Centre National D'Etudes Spatiales
- GRACE/GRACE-FO Data Release
- GRACE/GRACE-FO Data Product
    * GAA: non-tidal atmospheric correction  
    * GAB: non-tidal oceanic correction  
    * GAC: combined non-tidal atmospheric and oceanic correction  
    * GAD: GRACE/GRACE-FO ocean bottom pressure product  
    * GSM: corrected monthly GRACE/GRACE-FO static field product
- GRACE/GRACE-FO Date Range

In [ ]:
# dropdown menu for setting processing center
# CSR: University of Texas Center for Space Research  
# GFZ: German Research Centre for Geosciences (GeoForschungsZentrum)
# JPL: Jet Propulsion Laboratory    
# CNES: French Centre National D'Etudes Spatiales
proc_list = ['CSR', 'GFZ', 'JPL', 'CNES']
proc_default = 'CSR'
procDropdown = widgets.Dropdown(
    options=proc_list,
    value=proc_default,
    description='Center:',
    disabled=False,
)

# dropdown menu for setting data release
drel_list = ['RL04', 'RL05', 'RL06']
drel_default = 'RL06'
drelDropdown = widgets.Dropdown(
    description='Release:',
    options=drel_list,
    value=drel_default,
    disabled=False,
)

# dropdown menu for setting data product
# GAA: non-tidal atmospheric correction  
# GAB: non-tidal oceanic correction  
# GAC: combined non-tidal atmospheric and oceanic correction  
# GAD: GRACE/GRACE-FO ocean bottom pressure product  
# GSM: corrected monthly GRACE/GRACE-FO static field product
dset_list = ['GAC', 'GAD', 'GSM']
dset_default = 'GSM'
dsetDropdown = widgets.Dropdown(
    description='Product:',
    options=dset_list,
    value=dset_default,    
    disabled=False,
)
    
# find available months for data product
total_months = grace_find_months(base_dir, procDropdown.value,
    drelDropdown.value, DSET=dsetDropdown.value)
# select months to run
# https://tsutterley.github.io/data/GRACE-Months.html
options=['{0:03d}'.format(m) for m in total_months['months']]
monthsSelect = widgets.SelectMultiple(
    options=options,
    value=options,
    description='Months:',
    disabled=False
)

# function for setting the data release
def set_release(sender):
    if (procDropdown.value == 'CNES'):
        drel_list = ['RL03', 'RL04', 'RL05']
        drel_default = 'RL05'
    else:
        drel_list = ['RL04', 'RL05', 'RL06']
        drel_default = 'RL06'
    drelDropdown.options=drel_list
    drelDropdown.value=drel_default

# function for setting the data product
def set_product(sender):
    if ((procDropdown.value == 'CNES') and (drelDropdown.value == 'RL01')):
        dset_list = ['GAC', 'GSM']
    elif ((procDropdown.value == 'CNES') and drelDropdown.value in ('RL02','RL03','RL05')):
        dset_list = ['GAA', 'GAB', 'GSM']
    elif ((procDropdown.value == 'CNES') and (drelDropdown.value == 'RL04')):
        dset_list = ['GSM']
    elif (procDropdown.value == 'CSR'):
        dset_list = ['GAC', 'GAD', 'GSM']
    else:
        dset_list = ['GAA', 'GAB', 'GAC', 'GAD', 'GSM']
    dsetDropdown.options=dset_list
    dsetDropdown.value=dset_default
    
# function for updating the available month
def update_months(sender):
    total_months = grace_find_months(base_dir, procDropdown.value,
        drelDropdown.value, DSET=dsetDropdown.value)
    options=['{0:03d}'.format(m) for m in total_months['months']]
    monthsSelect.options=options
    monthsSelect.value=options

# watch widgets for changes
procDropdown.observe(set_release)
drelDropdown.observe(set_product)
procDropdown.observe(set_product)
procDropdown.observe(update_months)
drelDropdown.observe(update_months)

# display widgets for setting GRACE/GRACE-FO parameters
widgets.VBox([procDropdown,drelDropdown,dsetDropdown,monthsSelect])

### Set Parameters for Reading GRACE/GRACE-FO Data
These parameters describe processing steps and corrections to be applied when reading the GRACE/GRACE-FO data

- Maximum Degree and Order
- Geocenter product (Degree 1)
- Oblateness product (<i>C</i><sub>20</sub>)
- Low Degree Zonals (<i>C</i><sub>30</sub>)
- Pole Tide Correction from [Wahr et al. (2015)](https://doi.org/10.1002/2015JB011986)  
- Atmospheric Correction as described in [Fagiolini et al. (2015)](https://doi.org/10.1093/gji/ggv276)  

#### Geocenter
Measurements of time-variable gravity from the Gravity Recovery and Climate Experiment (GRACE) and the GRACE Follow-On (GRACE-FO) missions are set in a center of mass (CM) reference frame, in which the total degree one variations are inherently zero.
The individual contributions to degree one variations in the CM reference frame, such as from oceanic processes or terrestrial water storage change, are not necessarily zero ([Wahr et al., 1998](https://doi.org/10.1029/98JB02844)).
Applications set in a center of figure (CF) reference frame, such as the recovery of mass variations of the oceans, hydrosphere and cryosphere, require the inclusion of degree one terms to be fully accurate ([Swenson et al., 2008](https://doi.org/10.1029/2007JB005338)).
The geocenter options above can select the degree one product to include with the GRACE/GRACE-FO derived harmonics of degree two and higher.
There are options for using measurements from satellite laser ranging ([Cheng, 2013](https://doi.org/10.1007/978-3-642-32998-2_4)) and calculations from time-variable gravity and ocean model outputs ([Swenson et al., 2008](https://doi.org/10.1029/2007JB005338)).

#### Low Degree Zonals
For both GRACE and GRACE-FO, there have been operational issues that have affected the quality of the time-variable gravity fields.
During the late stages of the GRACE mission, procedures were enacted to preserve the battery life of the GRACE satellites and extend the mission lifetime.
This included turning off the accelerometer and microwave ranging instrument (MWI) to reduce the battery load on the spacecraft during periods of low &beta; angle when solar input to the spacecraft is lowest.
In late 2016, the accelerometer onboard GRACE-B was permanently powered down to help maintain the operation of the ranging instrument.
For the Follow-On mission, an anomaly shortly after the launch of GRACE-FO caused a malfunction and the accelerometer onboard GRACE-FO 2 (GF2) has been operating in a less optimal Large-Range-Mode.
For both of these cases, the GRACE/GRACE-FO processing centers have developed independent methods to spatiotemporally transplant the accelerometer data retrieved from GRACE-A to GRACE-B and from GF1 to GF2 ([Bandikova et al., 2019](https://doi.org/10.1016/j.asr.2019.05.021)).
These single-accelerometer months for both GRACE and GRACE-FO contain significantly more noise, particularly the low-degree zonal harmonics (particularly <i>C</i><sub>20</sub> and <i>C</i><sub>30</sub> but possibly <i>C</i><sub>40</sub> and <i>C</i><sub>50</sub>).
<i>C</i><sub>20</sub> has also been difficult for GRACE and GRACE-FO to independently measure throughout both missions.
Measurements from satellite laser ranging (SLR) can provide an independent assessment for some low degree and order spherical harmonics ([Cheng and Ries, 2018](https://doi.org/10.1093/gji/ggx483); [Loomis et al., 2019](https://doi.org/10.1029/2019GL082929); [Loomis et al., 2020](https://doi.org/10.1029/2019GL085488)).
The <i>C</i><sub>20</sub> and <i>C</i><sub>30</sub> options above can select the SLR low degree zonal product to use as a replacement for the GRACE/GRACE-FO derived estimates.

#### Corrections
Prior to GRACE/GRACE-FO Release-6, corrections needed to be applied to compensate for long-period signals in the pole tide that were contaminating the <i>C</i><sub>21</sub> and <i>S</i><sub>21</sub> harmonics ([Wahr et al., 2015](https://doi.org/10.1002/2015JB011986)), as well as for discontinuities in the atmospheric de-aliasing product that were introduced with upgrades in the ECMWF weather prediction model ([Fagiolini et al., 2015](https://doi.org/10.1093/gji/ggv276)).  The Pole Tide and Atmospheric corrections do not need to be applied to the Release-6 data. 

In [ ]:
# set the spherical harmonic truncation parameters
lmax_default = {}
lmax_default['CNES'] = {'RL01':50,'RL02':50,'RL03':80}
# CSR RL04/5/6 at LMAX 60
lmax_default['CSR'] = {'RL04':60,'RL05':60,'RL06':60}
# GFZ RL04/5 at LMAX 90
# GFZ RL06 at LMAX 60
lmax_default['GFZ'] = {'RL04':90,'RL05':90,'RL06':60}
# JPL RL04/5/6 at LMAX 60
lmax_default['JPL'] = {'RL04':60,'RL05':60,'RL06':60}
# text entry for spherical harmonic degree
lmaxText = widgets.BoundedIntText(
    min=0,
    max=lmax_default[procDropdown.value][drelDropdown.value],
    value=lmax_default[procDropdown.value][drelDropdown.value],
    step=1,
    description='<i>&#8467;</i><sub>max</sub>:',
    disabled=False
)

# text entry for spherical harmonic order
mmaxText = widgets.BoundedIntText(
    min=0,
    max=lmax_default[procDropdown.value][drelDropdown.value],
    value=lmax_default[procDropdown.value][drelDropdown.value],
    step=1,    
    description='<i>m</i><sub>max</sub>:',
    disabled=False
)

# dropdown menu for setting geocenter
# Tellus: GRACE/GRACE-FO TN-13 from PO.DAAC
#    https://grace.jpl.nasa.gov/data/get-data/geocenter/
# SLR: satellite laser ranging from CSR
#    ftp://ftp.csr.utexas.edu/pub/slr/geocenter/
# SLF: Sutterley and Velicogna, Remote Sensing (2019)
#    https://www.mdpi.com/2072-4292/11/18/2108
geocenter_list = ['[none]', 'Tellus', 'SLR', 'SLF']
geocenter_default = 'SLF' if (dsetDropdown.value == 'GSM') else '[none]'
geocenterDropdown = widgets.Dropdown(
    options=geocenter_list,
    value=geocenter_default,
    description='Geocenter:',
    disabled=False,
)

# SLR C20
C20_list = ['[none]','CSR','GSFC']
C20_default = 'GSFC' if (dsetDropdown.value == 'GSM') else '[none]'
C20Dropdown = widgets.Dropdown(
    options=C20_list,
    value=C20_default,
    description='SLR C20:',
    disabled=False,
)

# SLR C30
C30_list = ['[none]','CSR','GSFC']
C30_default = 'GSFC' if (dsetDropdown.value == 'GSM') else '[none]'
C30Dropdown = widgets.Dropdown(
    options=C30_list,
    value=C30_default,
    description='SLR C30:',
    disabled=False,
)

# Pole Tide Drift (Wahr et al., 2015) for Release-5
poletide_default = True if ((drelDropdown.value == 'RL05')
    and (dsetDropdown.value == 'GSM')) else False
poletideCheckbox = widgets.Checkbox(
    value=poletide_default,
    description='Pole Tide Corrections',
    disabled=False
)


# ECMWF Atmospheric Jump Corrections for Release-5
atm_default = True if (dsetDropdown.value == 'RL05') else False
atmCheckbox = widgets.Checkbox(
    value=atm_default,
    description='ATM Corrections',
    disabled=False
)

# functions for setting the spherical harmonic truncation
def set_SHdegree(sender):
    lmaxText.max=lmax_default[procDropdown.value][drelDropdown.value]
    lmaxText.value=lmax_default[procDropdown.value][drelDropdown.value]

def set_SHorder(sender):
    mmaxText.max=lmaxText.value
    mmaxText.value=lmaxText.value

# functions for setting pole tide and atmospheric corrections for Release-5
def set_pole_tide(sender):
    poletideCheckbox.value = True if ((drelDropdown.value == 'RL05')
        and (dsetDropdown.value == 'GSM')) else False

def set_atm_corr(sender):
    atmCheckbox.value = True if (drelDropdown.value == 'RL05') else False

# watch processing center widget for changes
procDropdown.observe(set_SHdegree)
# watch data release widget for changes
drelDropdown.observe(set_SHdegree)
drelDropdown.observe(set_pole_tide)
drelDropdown.observe(set_atm_corr)
# watch data product widget for changes
dsetDropdown.observe(set_pole_tide)
# watch spherical harmonic degree widget for changes
lmaxText.observe(set_SHorder)
        
# display widgets for setting GRACE/GRACE-FO read parameters
widgets.VBox([lmaxText,mmaxText,geocenterDropdown,
    C20Dropdown,C30Dropdown,poletideCheckbox,atmCheckbox])

### Read GRACE/GRACE-FO data
This step extracts the parameters chosen above and then reads the GRACE/GRACE-FO data applying the specified procedures  

In [ ]:
# extract values from widgets
PROC = procDropdown.value
DREL = drelDropdown.value
DSET = dsetDropdown.value
months = [int(m) for m in monthsSelect.value]
LMAX = lmaxText.value
MMAX = mmaxText.value
DEG1 = geocenterDropdown.value
SLR_C20 = C20Dropdown.value
SLR_C30 = C30Dropdown.value
POLE_TIDE = poletideCheckbox.value
ATM = atmCheckbox.value

# read GRACE/GRACE-FO data for parameters
start_mon = np.min(months)
end_mon = np.max(months)
missing = sorted(set(np.arange(start_mon,end_mon+1)) - set(months))
Ylms = grace_input_months(base_dir, PROC, DREL, DSET,
    LMAX, start_mon, end_mon, missing, SLR_C20, DEG1, MMAX=MMAX,
    SLR_C30=SLR_C30, POLE_TIDE=POLE_TIDE, ATM=ATM)
# create harmonics object and remove mean
GRACE_Ylms = harmonics().from_dict(Ylms)
GRACE_Ylms.mean(apply=True)
# directory of specific GRACE/GRACE-FO product
grace_dir = Ylms['directory']
# string denoting specific corrections and data used
grace_str = Ylms['title']
# number of time steps
nt = len(months)
# flag for spherical harmonic order
order_str = 'M{0:d}'.format(MMAX) if (MMAX != LMAX) else ''

### Set Parameters to Convert to Spatial Maps
These parameters specify corrections and filtering steps for converting to the spatial domain at a specified grid spacing  

- GIA Correction  
- Remove Specific Harmonic Fields  
- Redistribute Removed Fields over the Ocean  
- Gaussian Smoothing Radius in kilometers  
- Filter (destripe) harmonics [(Swenson and Wahr, 2006)](https://doi.org/10.1029/2005GL025285)  
- Spatial degree spacing  
- Spatial degree interval  
    1) (-180:180,90:-90)  
    2) (degree spacing)/2  
- Output spatial units  
    1) equivalent water thickness (cm)  
    2) geoid height (mm)  
    3) elastic crustal deformation (mm)  
    4) gravitational perturbation (&mu;Gal)  
    5) equivalent surface pressure (Pa)  
- Output spatial format  
    1) netCDF4  
    2) HDF5  

#### Geophysical Leakage
Gravity measurements from GRACE and GRACE-FO are global, near-monthly and are directly related to changes in mass.
Several mass transport processes can occur concurrently for a given region, which means that the total time-dependent geopotential from GRACE/GRACE-FO can relate to multiple time-varying components ([Wahr et al., 1998](https://doi.org/10.1029/98JB02844)).
These mass transport processes include but are not limited to terrestrial water storage, glacier and ice sheet mass, atmospheric and oceanic circulation and geodynamic processes.
In order to isolate the mass change of a single process, each of the other processes needs to be independently estimated and removed from the GRACE/GRACE-FO data.
Uncertainties in the components removed from the GRACE/GRACE-FO data will directly impact the precision of the final mass balance estimate.

#### Filtering
The GRACE/GRACE-FO coefficients are impacted by random spherical harmonic errors that increase as a function of spherical harmonic degree ([Wahr et al., 1998](https://doi.org/10.1029/98JB02844)).
The impact of these errors can be reduced using Gaussian averaging functions as described in  [Jekeli, (1981)](http://www.geology.osu.edu/~jekeli.1/OSUReports/reports/report_327.pdf).
GRACE/GRACE-FO coefficients are also impacted by correlated north/south "striping" errors, which can be spectrally filtered following [Swenson and Wahr (2006)](https://doi.org/10.1029/2005GL025285).

#### Units
Spatial fields of surface mass density can be estimated from sets of spherical harmonics if we assume that the mass redistributions are concentrated within a thin layer (thickness &#x226a; horizontal resolution) after correcting for glacial isostatic adjustment ([Wahr et al., 1998](https://doi.org/10.1029/98JB02844)).
However, we additionally need to compensate for the Earth's elastic yielding to surface load changes, which induce density anomalies at depth ([Wahr et al., 1998](https://doi.org/10.1029/98JB02844)).
This program accounts for the elastic deformation of the solid Earth using load Love numbers calculated by [Han and Wahr (1995)](https://doi.org/10.1111/j.1365-246X.1995.tb01819.x) with parameters from the Preliminary reference Earth model (PREM).
This program can output monthly spatial fields in a few different units (mass, deformation, and gravity).
- Equivalent water thickness
$$\Delta\sigma(\theta,\phi) = \frac{a\rho_{e}}{3\rho_{w}}\sum_{l=1}^{l_{max}}\sum_{m=0}^l\frac{2l+1}{1+k_l}\tilde{P}_{lm}(\cos\theta)\left[\Delta \tilde{C}_{lm}\cos{m\phi}+\Delta \tilde{S}_{lm}\sin{m\phi}\right]$$
- Geoid height
$$\Delta N(\theta,\phi) = a\sum_{l=1}^{l_{max}}\sum_{m=0}^l \tilde{P}_{lm}(\cos\theta)\left[\tilde{C}_{lm}\cos{m\phi}+ \tilde{S}_{lm}\sin{m\phi}\right]$$
- Vertical elastic crustal deformation  
$$\Delta z(\theta,\phi) = a\sum_{l=1}^{l_{max}}\sum_{m=0}^l\frac{h_l}{1+k_l}\tilde{P}_{lm}(\cos\theta)\left[\Delta \tilde{C}_{lm}\cos{m\phi}+\Delta \tilde{S}_{lm}\sin{m\phi}\right]$$

    * <i>a</i>: average radius of the Earth  
    * <i>&rho;<sub>e</sub></i>: average density of the Earth  
    * <i>&rho;<sub>w</sub></i>: density of water  
    * <i>h<sub>l</sub></i>, <i>k<sub>l</sub></i>: Load Love numbers of degree <i>l</i> 
    * <i>P<sub>lm</sub></i>: Fully normalized Legendre polynomials of degree <i>l</i> and order <i>m</i> 
    * <i>C<sub>lm</sub></i>, <i>S<sub>lm</sub></i>: Cosine and sine spherical harmonics of degree <i>l</i> and order <i>m</i> 
    * <i>&theta;</i>, <i>&phi;</i>: Colatitude and longitude in radians


In [ ]:
# set the GIA file
# files come in different formats depending on the group
giaText = widgets.Text(
    value='',
    description='GIA File:',
    disabled=False
)

# dropdown menu for setting GIA model
# IJ05-R2: Ivins R2 GIA Models
# W12a: Whitehouse GIA Models
# SM09: Simpson/Milne GIA Models
# ICE6G: ICE-6G GIA Models
# Wu10: Wu (2010) GIA Correction
# AW13-ICE6G: Geruo A ICE-6G GIA Models
# Caron: Caron JPL GIA Assimilation
# ICE6G-D: ICE-6G Version-D GIA Models
# ascii: GIA reformatted to ascii
# netCDF4: GIA reformatted to netCDF4
# HDF5: GIA reformatted to HDF5
gia_list = ['[None]','IJ05-R2','W12a','SM09','ICE6G',
    'Wu10','AW13-ICE6G','Caron','ICE6G-D',
    'ascii','netCDF4','HDF5']
gia_default = '[None]'
giaDropdown = widgets.Dropdown(
    options=gia_list,
    value=gia_default,
    description='GIA Type:',
    disabled=False,
)

# set the files to be removed
removeText = widgets.Text(
    value='',
    description='Rem. Files:',
    disabled=False
)

# dropdown menu for setting remove file type
# netCDF4: single netCDF4 file
# HDF5: single HDF5 file
# index (ascii): index of monthly ascii files
# index (netCDF4): index of monthly netCDF4 files
# index (HDF5): index of monthly HDF5 files
remove_list = ['[None]','netCDF4','HDF5',
    'index (ascii)','index (netCDF4)','index (HDF5)']
remove_default = '[None]'
removeDropdown = widgets.Dropdown(
    options=remove_list,
    value=remove_default,
    description='Rem. Type:',
    disabled=False,
)

# redestribute removed file mass over the ocean 
redistributeCheckbox = widgets.Checkbox(
    value=False,
    description='Redistribute Removed',
    disabled=False
)

# path to land-sea mask for ocean redistribuion
landmaskText = widgets.Text(
    value='',
    description='Mask File:',
    disabled=False
)

# text entry for Gaussian Smoothing Radius in km
gaussianText = widgets.BoundedFloatText(
    value=0,
    min=0,
    max=1000.0,
    step=50,
    description='Gaussian:',
    disabled=False
)

# Destripe Spherical Harmonics
destripeCheckbox = widgets.Checkbox(
    value=False,
    description='Destripe',
    disabled=False
)

# text entry for Degree Spacing
spacingText = widgets.BoundedFloatText(
    value=1.0,
    min=0,
    max=360.0,
    step=0.5,
    description='Spacing:',
    disabled=False
)

# dropdown menu for setting output data format
interval_list = ['(-180:180,90:-90)', '(Degree spacing)/2']
interval_default = '(Degree spacing)/2'
intervalDropdown = widgets.Dropdown(
    options=interval_list,
    value=interval_default,
    description='Interval:',
    disabled=False,
)

# dropdown menu for setting units
# 1: cm of water thickness
# 2: mm of geoid height
# 3: mm of elastic crustal deformation
# 4: microGal gravitational perturbation
# 5: millibar of equivalent surface pressure
unit_list = ['cmwe', 'mmGH', 'mmCU', u'\u03BCGal', 'mbar']
unit_label = ['cm', 'mm', 'mm', u'\u03BCGal', 'mb']
unit_name = ['Equivalent Water Thickness', 'Geoid Height',
    'Elastic Crustal Uplift', 'Gravitational Undulation',
    'Equivalent Surface Pressure']
unit_default = 'cmwe'
unitsDropdown = widgets.Dropdown(
    options=unit_list,
    value=unit_default,
    description='Units:',
    disabled=False,
)

# dropdown menu for setting output data format
format_list = ['[None]','netCDF4', 'HDF5']
format_default = '[None]'
formatDropdown = widgets.Dropdown(
    options=format_list,
    value=format_default,
    description='Format:',
    disabled=False,
)

# display widgets for setting GRACE/GRACE-FO read parameters
widgets.VBox([giaText,giaDropdown,removeText,removeDropdown,
    redistributeCheckbox,landmaskText,gaussianText,
    destripeCheckbox,spacingText,intervalDropdown,
    unitsDropdown,formatDropdown])

### Convert GRACE/GRACE-FO harmonics to the spatial domain
This step extracts the parameters chosen above and then converts the GRACE/GRACE-FO harmonics to the spatial domain applying the specified corrections and filtering procedures

- Set output grid domain  
- Calculate Fully-Normalized Legendre Polynomials  
- Read GIA model for correcting GRACE/GRACE-FO data  
- Read harmonics to be removed from the GRACE/GRACE-FO data  
- Calculate coefficients for converting to the output units  
- Convert from the spherical harmonic domain into the spatial domain  
- Output the monthly spatial maps to netCDF4 or HDF5  

In [ ]:
# Output spatial data
grid = spatial()
grid.time = np.copy(GRACE_Ylms.time)
grid.month = np.copy(GRACE_Ylms.month)

# Output degree spacing
dlon = spacingText.value
dlat = spacingText.value
# Output Degree Interval
INTERVAL = intervalDropdown.index + 1
if (INTERVAL == 1):
    # (-180:180,90:-90)
    nlon = np.int((360.0/dlon)+1.0)
    nlat = np.int((180.0/dlat)+1.0)
    grid.lon = -180 + dlon*np.arange(0,nlon)
    grid.lat = 90.0 - dlat*np.arange(0,nlat)
elif (INTERVAL == 2):
    # (Degree spacing)/2
    grid.lon = np.arange(-180+dlon/2.0,180+dlon/2.0,dlon)
    grid.lat = np.arange(90.0-dlat/2.0,-90.0-dlat/2.0,-dlat)
    nlon = len(grid.lon)
    nlat = len(grid.lat)
# update spacing and dimensions
grid.update_spacing()
grid.update_extents()
grid.update_dimensions()

# Computing plms for converting to spatial domain
theta = (90.0-grid.lat)*np.pi/180.0
PLM,dPLM = plm_holmes(LMAX,np.cos(theta))

# read load love numbers file
# PREM outputs from Han and Wahr (1995)
# https://doi.org/10.1111/j.1365-246X.1995.tb01819.x
love_numbers_file = get_data_path(['data','love_numbers'])
header = 2
columns = ['l','hl','kl','ll']
# LMAX of load love numbers from Han and Wahr (1995) is 696.
# from Wahr (2007) linearly interpolating kl works
# however, as we are linearly extrapolating out, do not make
# LMAX too much larger than 696
# read arrays of kl, hl, and ll Love Numbers
hl,kl,ll = read_love_numbers(love_numbers_file, LMAX=LMAX, HEADER=header,
    COLUMNS=columns, REFERENCE='CF', FORMAT='tuple')

# read GIA data
GIA = giaDropdown.value
GIA_Ylms_rate = read_GIA_model(giaText.value, GIA=GIA, LMAX=LMAX, MMAX=MMAX)
gia_str = '' if (GIA == '[None]') else '_{0}'.format(GIA_Ylms_rate['title'])
# calculate the monthly mass change from GIA
GIA_Ylms = GRACE_Ylms.zeros_like()
GIA_Ylms.time[:] = np.copy(GRACE_Ylms.time)
GIA_Ylms.month[:] = np.copy(GRACE_Ylms.month)
# monthly GIA calculated by gia_rate*time elapsed
# finding change in GIA each month
for t in range(nt):
    GIA_Ylms.clm[:,:,t] = GIA_Ylms_rate['clm']*(GIA_Ylms.time[t]-2003.3)
    GIA_Ylms.slm[:,:,t] = GIA_Ylms_rate['slm']*(GIA_Ylms.time[t]-2003.3)

# if redistributing removed mass over the ocean
if redistributeCheckbox.value:
    # read Land-Sea Mask and convert to spherical harmonics
    LSMASK = os.path.expanduser(landmaskText)
    ocean_Ylms = ocean_stokes(LSMASK,LMAX,MMAX=MMAX,LOVE=(hl,kl,ll))
    
# read data to be removed from GRACE/GRACE-FO monthly harmonics
remove_Ylms = GRACE_Ylms.zeros_like()
remove_Ylms.time[:] = np.copy(GRACE_Ylms.time)
remove_Ylms.month[:] = np.copy(GRACE_Ylms.time)
# If there are files to be removed from the GRACE/GRACE-FO data
remove_files = removeText.value.split(',') if removeText.value else []
# for each file separated by commas
for f in remove_files:
    if (removeDropdown.value == 'netCDF4'):
        # read netCDF4 file
        Ylms = harmonics().from_netCDF4(f)
    elif (removeDropdown.value == 'HDF5'):
        # read HDF5 file
        Ylms = harmonics().from_HDF5(f)
    elif (removeDropdown.value == 'index (ascii)'):
        # read index of ascii files
        Ylms = harmonics().from_index(f,'ascii')
    elif (removeDropdown.value == 'index (netCDF4)'):
        # read index of netCDF4 files
        Ylms = harmonics().from_index(f,'netCDF4')
    elif (removeDropdown.value == 'index (HDF5)'):
        # read index of HDF5 files
        Ylms = harmonics().from_index(f,'HDF5')
    # reduce to months of interest and truncate to range
    Ylms = Ylms.subset(months).truncate(LMAX,mmax=MMAX)
    # redistribute removed mass over the ocean
    if redistributeCheckbox.value:
        # calculate ratio between total removed mass and
        # a uniformly distributed cm of water over the ocean
        ratio = Ylms.clm[0,0,:]/ocean_Ylms['clm'][0,0]
        # for each spherical harmonic
        for m in range(0,MMAX+1):
            for l in range(m,LMAX+1):
                # remove the ratio*ocean Ylms from Ylms
                Ylms.clm[l,m,:]-=ratio*ocean_Ylms['clm'][l,m]
                Ylms.slm[l,m,:]-=ratio*ocean_Ylms['slm'][l,m]
    # add the harmonics to be removed to the total
    remove_Ylms.add(Ylms)

# gaussian smoothing radius in km (Jekeli, 1981)
RAD = gaussianText.value
if (RAD != 0):
    wt = 2.0*np.pi*gauss_weights(RAD,LMAX)
    gw_str = '_r{0:0.0f}km'.format(RAD)
else:
    # else = 1
    wt = np.ones((LMAX+1))
    gw_str = ''

# destriping the GRACE/GRACE-FO harmonics
ds_str = '_FL' if destripeCheckbox.value else ''

# Setting units factor for output
UNITS = unitsDropdown.index + 1
# dfactor is the degree dependent coefficients
# for specific spherical harmonic output units
if (UNITS == 1):
    # 1: cmwe, centimeters water equivalent
    dfactor = units(lmax=LMAX).harmonic(hl,kl,ll).cmwe
elif (UNITS == 2):
    # 2: mmGH, millimeters geoid height
    dfactor = units(lmax=LMAX).harmonic(hl,kl,ll).mmGH
elif (UNITS == 3):
    # 3: mmCU, millimeters elastic crustal deformation
    dfactor = units(lmax=LMAX).harmonic(hl,kl,ll).mmCU
elif (UNITS == 4):
    # 4: micGal, microGal gravity perturbations
    dfactor = units(lmax=LMAX).harmonic(hl,kl,ll).microGal
elif (UNITS == 5):
    # 5: mbar, millibar equivalent surface pressure
    dfactor = units(lmax=LMAX).harmonic(hl,kl,ll).mbar

# converting harmonics to truncated, smoothed coefficients in units
# combining harmonics to calculate output spatial fields
# output spatial grid
grid.data = np.zeros((nlat,nlon,nt))
for i,grace_month in enumerate(GRACE_Ylms.month):
    # GRACE/GRACE-FO harmonics for time t
    # and monthly files to be removed
    if destripeCheckbox.value:
        Ylms = GRACE_Ylms.index(i).destripe()
        Ylms.subtract(remove_Ylms.index(i).destripe())
    else:
        Ylms = GRACE_Ylms.index(i)
        Ylms.subtract(remove_Ylms.index(i))
    # Remove GIA rate for time
    Ylms.subtract(GIA_Ylms.index(i))
    # smooth harmonics and convert to output units
    Ylms.convolve(dfactor*wt)
    # convert spherical harmonics to output spatial grid
    grid.data[:,:,i] = harmonic_summation(Ylms.clm, Ylms.slm,
        grid.lon, grid.lat, LMAX=LMAX, MMAX=MMAX, PLM=PLM).T

# output to netCDF4 or HDF5
file_format = '{0}_{1}_{2}{3}{4}_{5}_L{6:d}{7}{8}{9}_{10:03d}-{11:03d}.{12}'
if (formatDropdown.value == 'netCDF4'):
    FILE = file_format.format(PROC,DREL,DSET,gia_str,grace_str,
        unit_list[UNITS-1],LMAX,order_str,gw_str,ds_str,
        months[0],months[-1],'nc')
    grid.to_netCDF4(os.path.join(grace_dir,FILE), varname='z',
        units=unit_list[UNITS-1], longname=unit_name[UNITS-1],
        title='GRACE/GRACE-FO Spatial Data', date=True)
elif (formatDropdown.value == 'HDF5'):
    FILE = file_format.format(PROC,DREL,DSET,gia_str,grace_str,
        unit_list[UNITS-1],LMAX,order_str,gw_str,ds_str,
        months[0],months[-1],'H5')
    grid.to_HDF5(os.path.join(grace_dir,FILE), varname='z',
        units=unit_list[UNITS-1], longname=unit_name[UNITS-1],
        title='GRACE/GRACE-FO Spatial Data', date=True)

### Set parameters for creating animation
This step specifies the [colormap](https://matplotlib.org/gallery/color/colormap_reference.html), the normalization for the contour plot colors and the step for the color bar.


In [ ]:
# slider for the plot min and max for normalization
vmin = np.min(grid.data).astype(np.int)
vmax = np.ceil(np.max(grid.data)).astype(np.int)
rangeSlider = widgets.IntRangeSlider(
    value=[vmin,vmax],
    min=vmin,
    max=vmax,
    step=1,
    description='Plot Range:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
)

# slider for steps in color bar
stepSlider = widgets.IntSlider(
    value=1,
    min=0,
    max=vmax-vmin,
    step=1,
    description='Plot Step:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
)

# all listed colormaps in matplotlib version
cmap_set = set(plt.cm.datad.keys()) | set(plt.cm.cmaps_listed.keys())
# colormaps available in this program
# (no reversed, qualitative or miscellaneous)
cmaps = {}
cmaps['Perceptually Uniform Sequential'] = ['viridis',
    'plasma','inferno','magma','cividis']
cmaps['Sequential'] = ['Greys','Purples','Blues','Greens',
    'Oranges','Reds','YlOrBr','YlOrRd','OrRd','PuRd','RdPu',
    'BuPu','GnBu','PuBu','YlGnBu','PuBuGn','BuGn','YlGn']
cmaps['Sequential (2)'] = ['binary','gist_yarg','gist_gray', 
    'gray','bone','pink','spring','summer','autumn','winter',
    'cool','Wistia','hot','afmhot','gist_heat','copper']
cmaps['Diverging'] = ['PiYG','PRGn','BrBG','PuOr','RdGy','RdBu',
    'RdYlBu','RdYlGn','Spectral','coolwarm', 'bwr','seismic']
cmaps['Cyclic'] = ['twilight','twilight_shifted','hsv']
# create list of available colormaps in program
cmap_list = []
for key,val in cmaps.items():
    cmap_list.extend(val)
# reduce colormaps to available in program and matplotlib
cmap_set &= set(cmap_list)
# dropdown menu for setting colormap
cmapDropdown = widgets.Dropdown(
    options=sorted(cmap_set),
    value='viridis',
    description='Colormap:',
    disabled=False,
)

# Reverse the colormap
cmapCheckbox = widgets.Checkbox(
    value=False,
    description='Reverse Colormap',
    disabled=False
)

# display widgets for setting GRACE/GRACE-FO plot parameters
widgets.VBox([rangeSlider,stepSlider,cmapDropdown,cmapCheckbox])

### Create animation of GRACE/GRACE-FO months

In [ ]:
fig, ax1 = plt.subplots(num=1, nrows=1, ncols=1, figsize=(10.375,6.625),
    subplot_kw=dict(projection=ccrs.PlateCarree()))

# levels and normalization for plot range
vmin,vmax = rangeSlider.value
levels = np.arange(vmin,vmax+stepSlider.value,stepSlider.value)
norm = colors.Normalize(vmin=vmin,vmax=vmax)
cmap_reverse_flag = '_r' if cmapCheckbox.value else ''
cmap = plt.cm.get_cmap(cmapDropdown.value + cmap_reverse_flag)
im = ax1.imshow(np.zeros((nlat,nlon)), interpolation='nearest',
    norm=norm, cmap=cmap, transform=ccrs.PlateCarree(),
    extent=grid.extent, origin='upper', animated=True)
ax1.coastlines('50m')

# add date label (year-calendar month e.g. 2002-01)
time_text = ax1.text(0.025, 0.025, '', transform=fig.transFigure,
    color='k', size=24, ha='left', va='baseline')

# Add horizontal colorbar and adjust size
# extend = add extension triangles to upper and lower bounds
# options: neither, both, min, max
# pad = distance from main plot axis
# shrink = percent size of colorbar
# aspect = lengthXwidth aspect of colorbar
cbar = plt.colorbar(im, ax=ax1, extend='both', extendfrac=0.0375,
    orientation='horizontal', pad=0.025, shrink=0.85,
    aspect=22, drawedges=False)
# rasterized colorbar to remove lines
cbar.solids.set_rasterized(True)
# Add label to the colorbar
cbar.ax.set_xlabel(unit_name[UNITS-1], labelpad=10, fontsize=24)
cbar.ax.set_ylabel(unit_label[UNITS-1], fontsize=24, rotation=0)
cbar.ax.yaxis.set_label_coords(1.045, 0.1)
# Set the tick levels for the colorbar
cbar.set_ticks(levels)
cbar.set_ticklabels(['{0:d}'.format(ct) for ct in levels])
# ticks lines all the way across
cbar.ax.tick_params(which='both', width=1, length=26, labelsize=24,
    direction='in')
    
# stronger linewidth on frame
ax1.spines['geo'].set_linewidth(2.0)
ax1.spines['geo'].set_capstyle('projecting')
# adjust subplot within figure
fig.subplots_adjust(left=0.02,right=0.98,bottom=0.05,top=0.98)
    
# animate frames
def animate_frames(i):
    # set image
    im.set_data(grid.data[:,:,i])
    # add date label (year-calendar month e.g. 2002-01)
    year = np.floor(grid.time[i]).astype(np.int)
    month = np.int((grid.month[i]-1) % 12) + 1
    time_text.set_text(u'{0:4d}\u2013{1:02d}'.format(year,month))

# set animation
anim = animation.FuncAnimation(fig, animate_frames, frames=nt)
%matplotlib inline
HTML(anim.to_jshtml())